In [1]:
# Imports necessary to acquire, prepare, explore, visualize, analyze, and model data

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env
import acquire
import prepare 

from tabulate import tabulate
from IPython.display import Markdown, display

from math import sqrt
from scipy import stats

import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr, spearmanr

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer

np.random.seed(123)

In [2]:
df = acquire.get_zillow_data()
# this is a user-defined function in acquire.py that pulls in selected data from the SQL database

### Let's see what we got 

In [ ]:
df.isna().sum().sum() /len(df)
# returns percentage of nulls in df

#### Lots of nulls, will want to deal w that in a moment

In [ ]:
df.info()

#### lots of columns, a lot of those tell us nothing and were just here for a join. let's get those keys and then toss those

In [ ]:
df.isna().sum() > (len(df)/2)

getting rid of columns with almost all nulls (more than 80% for now)

In [ ]:
df = df.dropna(thresh=df.shape[0]*0.2,how='all',axis=1)

In [ ]:
df.info()

looking at duplicate columns...

In [ ]:
df.calculatedbathnbr.value_counts() #can drop

In [ ]:
df.bathroomcnt.value_counts

In [ ]:
df = df.drop(columns=['calculatedbathnbr'])

In [ ]:
df.info()

In [ ]:
df.finishedsquarefeet12

In [ ]:
df[(df.calculatedfinishedsquarefeet != df.finishedsquarefeet12)] #drop

In [ ]:
df= df.drop(columns=['finishedsquarefeet12'])

In [ ]:
df.info()

In [ ]:
df = df.drop(columns = 'fullbathcnt')

In [ ]:
df.info()

In [ ]:
df.propertylandusetypeid.value_counts() #drop and land use code

In [ ]:
df = df.drop(columns = ['propertycountylandusecode', 'propertylandusetypeid'])

In [ ]:
df.info()

In [ ]:
df.heatingorsystemtypeid.value_counts()

In [ ]:
df.airconditioningtypeid.value_counts()

In [ ]:
df['airconditioningtypeid'] = df['airconditioningtypeid'].fillna(0)

In [ ]:
df['heatingorsystemtypeid'] = df['heatingorsystemtypeid'].fillna(0)

In [ ]:
df.id.nunique() #redundant, drop

In [ ]:
df.info()

In [ ]:
df.buildingqualitytypeid.value_counts() # too many nulls that can't be filled with something reasonable

In [ ]:
df= df.drop(columns=['buildingqualitytypeid', 'id', 'id.1'])

In [ ]:
df.info()

In [ ]:
df.garagecarcnt.value_counts()

In [ ]:
df[['garagecarcnt', 'garagetotalsqft']] = df[['garagecarcnt', 'garagetotalsqft']].fillna(0)

In [ ]:
df.info()

In [ ]:
df.poolcnt.value_counts()

In [ ]:
df['poolcnt'] = df['poolcnt'].fillna(0)

In [ ]:
df.propertyzoningdesc.isna().sum() #this could possible tell something, but too many nulls and not something worth MVP time right now, dropping

In [ ]:
df = df.drop(columns='propertyzoningdesc')

In [ ]:
df.info()

In [ ]:
(df.regionidneighborhood.value_counts() > 10).sum() # can't get most of these, will drop this and other location ids that I can't get info on for MVP

## [1,] "27080_Sherman Oaks"          
## [2,] "118208_South Los Angeles"    
## [3,] "268496_Southeast Los Angeles"
## [4,] "41466_Van Nuys"              
## [5,] "32059_Hollywood"             
## [6,] "275237_Roosevelt"

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [ ]:
df.regionidcity.value_counts()

In [ ]:
df = df.drop(columns=['regionidcity', 'regionidzip', 'regionidneighborhood'])

In [ ]:
df.info()

In [ ]:
df.roomcnt.value_counts() # so many 0s, and what are the other rooms? I don't know what this is so dropping rn

In [ ]:
df = df.drop(columns= 'roomcnt')

In [ ]:
df.info()

In [ ]:
df.unitcnt.value_counts() #useless, drop

In [ ]:
df = df.drop(columns=['unitcnt'])

In [ ]:
df.info()

In [ ]:
df.numberofstories.value_counts() #too many nulls, drop

In [ ]:
df = df.drop(columns= 'numberofstories')

In [ ]:
#we cannot use $ data as that'd be unknown for making predictions

In [ ]:
df = df.drop(columns = ['structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxvaluedollarcnt', 'taxamount'])

In [ ]:
df.info()

In [ ]:
df = df.drop(columns= 'assessmentyear')

In [ ]:
df.airconditioningdesc.value_counts() #no, I don't like this for now. none and also so many nulls. And then just yes. this is too ambigious. drop

In [ ]:
df = df.drop(columns=['airconditioningdesc', 'airconditioningtypeid'])

In [ ]:
df.heatingorsystemdesc.value_counts() #still too many nulls, I will drop for now bc we don't know where the nulls go

In [ ]:
df = df.drop(columns=['heatingorsystemdesc', 'heatingorsystemtypeid'])

In [ ]:
df.info()

In [ ]:
df = df.dropna()

In [ ]:
df.info()

In [ ]:
#renaming columns

df.rename(columns={'bedroomcnt': 'bedrooms',
                   'taxvaluedollarcnt': 'tax_value', 
                   'lotsizesquarefeet': 'lot_size', 
                   'bathroomcnt': 'bathrooms', 
                   'calculatedfinishedsquarefeet': 'square_feet', 
                   'yearbuilt': 'year_built',
                    'garagecarcnt': 'garages',
                    'garagetotalsqft':'garage_size',
                    'poolcnt': 'has_pool',
                   'transactiondate': 'transaction_date'
                  }, inplace=True)

In [ ]:
df.info()

In [ ]:
df['has_pool'] = df.has_pool.astype(bool)

In [ ]:
df.censustractandblock.head()

### handling outliers

In [ ]:
df = df[df.bathrooms >= 1]
    df= df[df.bathrooms <= 5]
    df = df[df.bedrooms >= 1]
    df = df[df.bedrooms <= 5]
    df = df[df.log_error < 0.5]
    df = df[df.log_error > (-0.31)]
    df = df[df.year_built >= 1910]
    df = df[df.square_feet >= 650]
    df = df[df.square_feet <= 5500]
    df = df[df.tax_value > 40000.0]
    df = df[df.tax_value < 3000000.0]

## DROP LIST

df = df[df.bathrooms >= 1]
    df= df[df.bathrooms <= 5]
    df = df[df.bedrooms >= 1]
    df = df[df.bedrooms <= 5]
    df = df[df.log_error < 0.5]
    df = df[df.log_error > (-0.31)]
    df = df[df.year_built >= 1910]
    df = df[df.square_feet >= 650]
    df = df[df.square_feet <= 5500]
    df = df[df.tax_value > 40000.0]
    df = df[df.tax_value < 3000000.0]
- df = df.dropna(thresh=df.shape[0]*0.8,how='all',axis=1)
- df = df.drop(columns=['calculatedbathnbr', 'finishedsquarefeet12', 'fullbathcnt', 'propertycountylandusecode', 'propertylandusetypeid', 'id', 'id.1', 'buildingqualitytypeid', 'propertyzoningdesc', 'regionidcity', 'regionidzip', 'regionidneighborhood', 'roomcnt', 'unitcnt', 'numberofstories', 'structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxvaluedollarcnt', 'taxamount', 'assessmentyear', 'airconditioningdesc', 'airconditioningtypeid', 'heatingorsystemdesc', 'heatingorsystemtypeid', 'regionidcounty', 'propertyzoningdesc', 'rawcensustractandblock', 'rawcensustractandblock'])


## Replacing Stuff
- NOT NEED df['heatingorsystemtypeid'] = df['heatingorsystemtypeid'].fillna(0)
- NOT NEED df['airconditioningtypeid'] = df['airconditioningtypeid'].fillna(0)
- df[['garagecarcnt', 'garagetotalsqft']] = df[['garagecarcnt', 'garagetotalsqft']].fillna(0)
- df['poolcnt'] = df['poolcnt'].fillna(0)

## Handling Nulls after dropping stuff
- dropping all nulls from DF since there is such a small percentage
- df = df.dropna()

## Renaming Stuff

df.rename(columns={'bedroomcnt': 'bedrooms',
                   'taxvaluedollarcnt': 'tax_value', 
                   'lotsizesquarefeet': 'lot_size', 
                   'bathroomcnt': 'bathrooms', 
                   'calculatedfinishedsquarefeet': 'square_feet', 
                   'yearbuilt': 'year_built',
                    'garagecarcnt': 'garages',
                    'garagetotalsqft':'garage_size',
                    'poolcnt': 'has_pool',
                    'logerror': 'log_error',
                   'transactiondate': 'transaction_date'
                  }, inplace=True)

## outliers
## this needs to be first
df = df[df.bathrooms >= 1]
    df= df[df.bathrooms <= 5]
    df = df[df.bedrooms >= 1]
    df = df[df.bedrooms <= 5]
    df = df[df.log_error < 0.5]
    df = df[df.log_error > (-0.31)]
    df = df[df.year_built >= 1910]
    df = df[df.square_feet >= 650]
    df = df[df.square_feet <= 5500]
    df = df[df.tax_value > 40000.0]
    df = df[df.tax_value < 3000000.0]

## data types

df['year_built'] = df['year_built'].astype('int')
    df['fips'] = df['fips'].astype('int')
    df['square_feet'] = df['square_feet'].astype('int')
    df['county'] = df['fips'].replace({6037: 'los_angeles', 6059: 'orange', 6111: 'ventura'})
    df['lot_size'] = df.lot_size.astype(int)
    df['garages'] = df.garages.astype(int)
df['garage_size'] = df.garage_sizd.astype(int)
df['has_pool'] = df.has_pool.astype(bool)

## Testing functions for prepare-

In [3]:
df = prepare.remove_outliers(df)

In [ ]:
df.info()

In [ ]:
df = df.dropna(thresh=df.shape[0]*0.2,how='all',axis=1)

In [4]:
df = prepare.handle_nulls(df)

In [ ]:
df.info()

In [5]:
df = prepare.rename_columns(df)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47892 entries, 0 to 52440
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   parcelid             47892 non-null  int64         
 1   bathrooms            47892 non-null  float64       
 2   bedrooms             47892 non-null  float64       
 3   square_feet          47892 non-null  int64         
 4   fips                 47892 non-null  int64         
 5   garages              47892 non-null  int64         
 6   garage_size          47892 non-null  int64         
 7   latitude             47892 non-null  float64       
 8   longitude            47892 non-null  float64       
 9   lot_size             47892 non-null  int64         
 10  has_pool             47892 non-null  bool          
 11  year_built           47892 non-null  int64         
 12  censustractandblock  47892 non-null  float64       
 13  log_error            47892 non-

In [7]:
df = prepare.prepare_locs(df)

In [8]:
df.head()

,parcelid,bathrooms,bedrooms,square_feet,fips,garages,garage_size,lot_size,has_pool,year_built,censustractandblock,log_error,transaction_date,county,latitude,longitude
0,14297519,3.5,4.0,3100,6059,2,633,4506,False,1998,6.059063e+13,0.025595,2017-01-01,orange,33.634931,-117.869207
1,17052889,1.0,2.0,1465,6111,1,0,12647,False,1967,6.111001e+13,0.055619,2017-01-01,ventura,34.449266,-119.281531
2,14186244,2.0,3.0,1243,6059,2,440,8432,True,1962,6.059022e+13,0.005383,2017-01-01,orange,33.886168,-117.823170
3,12177905,3.0,4.0,2376,6037,0,0,13038,True,1970,6.037300e+13,-0.103410,2017-01-01,los_angeles,34.245180,-118.240722
4,12095076,3.0,4.0,2962,6037,0,0,63000,True,1950,6.037461e+13,-0.001011,2017-01-01,los_angeles,34.145202,-118.179824


NameError: name 'train' is not defined